Comorasu Ana-Maria

Grupa 334

Tema 1 - Computer Vision

# Utilities

## Import Libraries

In [1]:
import cv2 as cv
import numpy as np
import math
import os
# import tensorflow
# from tensorflow import keras

In [6]:
# Constants - PLEASE CHANGE IF YOU WANT TO ON OTHER DATA
CLASIC_IMGS = 20
JIGSAW_IMGS = 40
BONUS = 'bonus'

## Directories and files

In [2]:
# * for reading images, call read_images function with either clasic or jigsaw and the number of images
def read_images(type: str, number: int):
    train_dir = './antrenare/' + type + '/'
    names = [train_dir + str(i).zfill(2) + '.jpg' for i in range(1, number + 1)]
    # names = [train_dir + str(i).zfill(2) + '.jpg' for i in [2, 7, 8, 10, 11, 15, 24, 30, 40]]
    training_imgs = [cv.imread(name) for name in names]
    training_imgs = [cv.resize(i, (0, 0), fx=0.25, fy=0.25) for i in training_imgs]
    return training_imgs

### For submission

In [3]:
def make_submission_dirs():
    os.mkdir('./fisiere_solutie')
    os.mkdir('./fisiere_solutie/clasic')
    os.mkdir('./fisiere_solutie/jigsaw')

### Show image

In [4]:
def show_image(title, image):
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

## Clasic and Jigsaw Images

In [7]:
clasic_data = read_images('clasic', CLASIC_IMGS)
jigsaw_data = read_images('jigsaw', JIGSAW_IMGS)

# Preprocess images

### Normalize image

In [8]:
# * Normalize image - remove shadows
def normalize_image(img):
    noise = cv.dilate(img, np.ones((7, 7), np.uint8))
    blur = cv.medianBlur(noise, 21)
    res = 255 - cv.absdiff(img, blur)
    no_shadow = cv.normalize(res, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX)
    return no_shadow

### Apply filters

#### Initial filters

In [9]:
def filter_image_v1(image):
    # varianta laborator
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    img_norm = normalize_image(image)
    img_gblur = cv.GaussianBlur(img_norm, (9, 9), 0)
    img_sharp = cv.addWeighted(img_norm, 1.2, img_gblur, -0.8, 0)

    _, thresh = cv.threshold(img_sharp, 20, 255, cv.THRESH_BINARY)
    
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.erode(thresh, kernel)

    inverted = cv.bitwise_not(thresh, 0)

    return inverted


def filter_image_v2(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    img_norm = normalize_image(image)
    img_gblur = cv.GaussianBlur(img_norm, (9, 9), 0)

    thresh = cv.adaptiveThreshold(img_gblur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 2)
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (2, 2))
    morph = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel)

    res = cv.dilate(morph, kernel, iterations=1)
    res = cv.erode(res, kernel)

    return res


def filter_image_v3(image):
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (2, 2))

    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    img_norm = normalize_image(image)
    img_gblur = cv.GaussianBlur(img_norm, (1, 1), 0)
    img_gblur = cv.medianBlur(img_gblur, 11)
    # show_image("median blur", img_gblur)

    thresh = cv.adaptiveThreshold(img_gblur, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 9, 12)
    # show_image("thresh", thresh)
    morph = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel)

    # # res = cv.dilate(morph, kernel, iterations=1)
    # res1 = cv.erode(morph, kernel, iterations=6)
    # res1 = cv.dilate(res1, kernel, iterations=4)
    # show_image("res1", res1)

    res2 = cv.erode(morph, kernel, iterations=1)
    res2 = cv.dilate(res2, kernel, iterations=2)
    # show_image("res2", res2)

    return res2

#### Warp filter

In [10]:
def warp_image(corners, image):
    corners = np.array(corners, dtype='float32')
    top_left, top_right, bottom_right, bottom_left = corners

    width = int(max([
        np.linalg.norm(top_right - bottom_right),
        np.linalg.norm(top_left - bottom_left),
        np.linalg.norm(bottom_right - bottom_left),
        np.linalg.norm(top_left - top_right)
    ]))

    mapping = np.array([[0, 0], [width - 1, 0], [width - 1, width - 1], [0, width - 1]], dtype='float32')
    matrix = cv.getPerspectiveTransform(corners, mapping)

    return cv.warpPerspective(image, matrix, (width, width)), matrix

### Find contours

In [11]:
def find_contours(image):
    edges =  cv.Canny(image, 150, 400)
    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0

    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    image_copy = cv.cvtColor(image.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),4,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),4,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),4,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),4,(0,0,255),-1)
    # show_image("detected corners", image_copy)

    return top_left, top_right, bottom_right, bottom_left

### Preprocess (combining previous functions)

In [12]:
def preprocess_task1(image):
    filtered_img = filter_image_v2(image)
    corners = find_contours(filtered_img)

    warped, _ = warp_image(corners, image)
    warped_processed = filter_image_v2(warped)
    return warped_processed


def preprocess_task2(image):
    filtered_img = filter_image_v2(image)
    corners = find_contours(filtered_img)

    warped, _ = warp_image(corners, image)
    warped_processed = filter_image_v3(warped)
    return warped_processed

### Get vertical and horizontal lines

In [13]:
def get_grid_line(image, type):
    img_cpy = image.copy()
    if type == 'h':
        s = img_cpy.shape[1]
    else:
        s = img_cpy.shape[0]
    size = s // 10

    if type == 'h':
        kernel = cv.getStructuringElement(cv.MORPH_RECT, (size, 1))
    else:
        kernel = cv.getStructuringElement(cv.MORPH_RECT, (1, size))
    
    img_cpy = cv.erode(img_cpy, kernel)
    img_cpy = cv.dilate(img_cpy, kernel)

    return img_cpy


def grid_lines(image):
    vertical = get_grid_line(image, 'v')
    horizontal = get_grid_line(image, 'h')

    return vertical, horizontal

### Draw the grid

In [14]:
def draw_grid_lines(img, lines):
    clone = img.copy()
    lines = np.squeeze(lines)
    for r, t in lines:
        a, b = np.cos(t), np.sin(t)
        x0, y0 = a * r, b * r
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * a)
        cv.line(clone, (x1, y1), (x2, y2), (255, 255, 255), 3)
    return clone

### Mask the grid

In [15]:
def grid_mask(vertical, horizontal):
    grid = cv.add(horizontal, vertical)
    grid = cv.adaptiveThreshold(grid, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 235, 2)
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
    grid = cv.dilate(grid, kernel, iterations=2)

    points = cv.HoughLines(grid, 0.3, np.pi/90, 200)
    lines = draw_grid_lines(grid, points)
    mask = cv.bitwise_not(lines)
    return mask

### Check if the cell is empty

In [16]:
def cell_is_empty(cell):
    cell = np.array(cell)
    avg_color_row = np.average(cell, axis=0)
    avg_color = np.average(avg_color_row, axis=0)
    return avg_color <= 6

### Get Sudoku board cells

In [17]:
def get_cells(image):
    cells = []
    width, _ = image.shape
    width_cell = width // 9
    i = 0
    for _ in range(9):
        cells.append([])
        j = 0
        for _ in range(9):
            cell = image[i : i + width_cell, j : j + width_cell]
            cells[-1].append(cell)
            j += width_cell
        i += width_cell
    return cells

# Solve tasks

### Load Model

In [ ]:
# model = keras.models.load_model("MNIST.h5")

## Task 1 result to write in file

In [19]:
# task 1
def task1(cells, type=''):
    res = ''
    for cell_line in cells:
        for curr_cell in cell_line:
            # * erode image
            cell = curr_cell.astype("float32")
            cell = cv.resize(cell[10 : -10, 10: -10], (28, 28))
            # cell = cv.erode(cell, (1, 1))
            # cell = np.reshape(cell, (1, 224, 224, 1))
            if cell_is_empty(cell):
                res += 'o'
            else:
                if type != BONUS:
                    res += 'x'
                # else:
                #     predictions = model.predict(tensorflow.image.grayscale_to_rgb(cell))
                #     digit = np.where(predictions == np.amax(predictions))
                #     res += str(digit[1][0])
                    
        res += '\n'
    return res.rstrip('\n')

### Solve task 1

In [24]:
def solve_task_1(type=''):
    # todo uncomment if the directories are not creaated
    make_submission_dirs()
    submit_clasic = './fisiere_solutie/clasic/'
    plus = ('_' + BONUS) if type == BONUS else ''
    submit_names = [submit_clasic + str(i) + plus + '_predicted.txt' for i in range(1, CLASIC_IMGS + 1)]
    i = 0
    # * already have clasic_data loaded
    for curr_image in clasic_data:
        # * warp and preprocess image
        warped = preprocess_task1(curr_image)

        # * create a mask
        vertical, horizontal = grid_lines(warped)
        mask = grid_mask(vertical, horizontal)
        show_image("mask", mask)

        # * remove the lines from the board
        board = cv.bitwise_and(warped, mask)
        show_image('board', board)

        cells = get_cells(board)
        result = task1(cells) if type != BONUS else task1(cells, BONUS)

        # * write result in file
        file_write = open(submit_names[i], 'w')
        file_write.write(result)
        i += 1
        


## Solve task 2

In [21]:
def task2(cells, colors, type=''):
    res = ''
    for i in range(9):
        for j in range(9):
            curr_cell = cells[i][j]
            cell = curr_cell.astype("float32")
            cell = cv.resize(cell[10 : -10, 10: -10], (28, 28))
            # cell = np.reshape(cell, (1, 224, 224, 1))
            res += str(colors[i][j])
            if cell_is_empty(cells[i][j]):
                res += 'o'
            else:
                if type != BONUS:
                    res += 'x'
                # else:
                #     predictions = model.predict(tensorflow.image.grayscale_to_rgb(cell))
                #     digit = np.where(predictions == np.amax(predictions))
                #     res += str(digit[1][0])
        res += '\n'
    return res.rstrip('\n')

### Graph class to solve islands problem

In [22]:
class Graph:
    def __init__(self) -> None:
        self.adj_list = []
        self.visited = [[-1 for _ in range(9)] for _ in range(9)]

    def number(self, line, columnn):
        return 9 * line + columnn

    def indices(self, num):
        return [num // 9, num % 9]

    def is_edge(self, edge) -> bool:
        return edge in self.adj_list

    def add_edge(self, node_a, node_b) -> None:
        if [node_a, node_b] not in self.adj_list:
            self.adj_list.append([node_a, node_b])
            self.adj_list.append([node_b, node_a])
    
    def connected_points(self, board, start, stop):
        # * vertically arranged
        if start[0] == stop[0]:
            part = board[start[1] + 15 : stop[1] - 15, start[0] - 15 : stop[0] + 15]
            avg = np.mean(part)
            return avg < 15
        # * horizontally arranged
        else:
            part = board[start[1] - 15 : stop[1] + 15, start[0] + 15 : stop[0] - 15]
            # show_image("part vert", part)
            avg = np.mean(part)
            return avg < 15

    def construct_graph_from_board(self, board):
        width = board.shape[0]
        cell_width = width // 9
        # search vertically first
        for i in range(1, 9):
            for j in range(0, 9):
                point_start = (i * cell_width, j * cell_width)
                point_stop = (i * cell_width, (j + 1) * cell_width)
                if self.connected_points(board, point_start, point_stop):
                    self.add_edge(self.number(j, i - 1), self.number(j, i))
        
        for i in range(1, 9):
            for j in range(0, 9):
                point_start = (j * cell_width, i * cell_width)
                point_stop = ((j + 1) * cell_width, i * cell_width)
                if self.connected_points(board, point_start, point_stop):
                    self.add_edge(self.number(i - 1, j), self.number(i, j))

    def transform_indices(self):
        new_list = []
        for [i, j] in self.adj_list:
            new_list.append([self.indices(i), self.indices(j)])
        self.adj_list = new_list

    def dfs(self, i, j, color):
        self.visited[i][j] = color
        for [[a_i, a_j], [b_i, b_j]] in self.adj_list:
            if a_i == i and a_j == j and self.visited[b_i][b_j] == -1:
                self.dfs(b_i, b_j, color)
            
    def solve_islands(self):
        color = 1
        for i in range(9):
            for j in range(9):
                if self.visited[i][j] == -1:
                    self.dfs(i, j, color)
                    color += 1
                    

### Task 2 solver

In [23]:
def solve_task_2(type=''):
    submit_jigsaw = './fisiere_solutie/jigsaw/'
    submit_names = [submit_jigsaw + str(i) + '_predicted.txt' for i in range(1, JIGSAW_IMGS + 1)]
    i = 0
    for curr_image in jigsaw_data:
        # * warp and preprocess image
        warped = preprocess_task2(curr_image)
        show_image("pentru linii", warped)

        # * lucrez cu warped image pentru muchiile grafului
        graph = Graph()
        graph.construct_graph_from_board(warped)
        graph.transform_indices()
        graph.solve_islands()

        # * preprocess again
        warped = preprocess_task1(curr_image)
        
        # * create a mask
        vertical, horizontal = grid_lines(warped)
        mask = grid_mask(vertical, horizontal)

        # * remove the lines from the board
        board = cv.bitwise_and(warped, mask)
        # show_image("board", board)
        cells = get_cells(board)
        result = task2(cells, graph.visited) if type != BONUS else task2(cells, graph.visited, BONUS)

        file_write = open(submit_names[i], 'w')
        file_write.write(result)
        i += 1
         

# Main program

In [25]:
def main():
    # * Solve and submit task 1
    solve_task_1()
    # * Solve and submit task 1 - BONUS
    # solve_task_1(BONUS)
    # * Solve an submit task 2
    solve_task_2()
    # * Solve and submit task 2 - BONUS
    # solve_task_2(BONUS)
    pass

if __name__=="__main__":
    main()